Stratified Subsampling the dataset from 13M to 400k


*   preserve both the balance ratio and the approval rate for each race



In [ ]:
seed = 1111


In [ ]:
# !pip install --upgrade xgboost
# !pip install catboost
# !pip install --upgrade numpy
# !pip install --upgrade pandas

# !pip install --upgrade scikit-learn
# !pip install --upgrade ipywidgets


import numpy as np  # This line should remain to import the correct version
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

In [ ]:


cols = ['conforming_loan_limit', 'derived_dwelling_category',
       'derived_race', 'applicant_sex', 'co-applicant_sex', 'action_taken', 'applicant_age',
       'applicant_age_above_62', 'co-applicant_age', 'co-applicant_age_above_62',
       'loan_type', 'loan_purpose', 'open-end_line_of_credit',
       'business_or_commercial_purpose', 'loan_amount', 'loan_to_value_ratio',
       'loan_term', 'property_value', 'occupancy_type', 'total_units', 'income',
       'debt_to_income_ratio', 'tract_population', 'tract_minority_population_percent',
       'ffiec_msa_md_median_family_income', 'tract_to_msa_income_percentage',
       'tract_owner_occupied_units', 'tract_one_to_four_family_homes',
       'tract_median_age_of_housing_units', 'negative_amortization',
        'other_nonamortizing_features', 'applicant_credit_score_type',
        'co-applicant_credit_score_type', 'interest_rate', 'origination_charges', 'total_loan_costs']


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
num_samples = 50000
# num_samples = 5000
# num_samples = 5000//3
N = 0
ratio = None
df_list = []
df_val_list = []
df_test_list = []

for year in ['2018', '2019', '2020', '2021', '2022', '2023']:
# for year in ['2018']:
    df = pd.read_csv(f"/content/drive/MyDrive/math6912/data/state_CA_{year}.csv")
    N += df.shape[0]
    print(year, df.shape)
    df = df[cols]
    df = df[df['derived_race'].isin(['White', 'Black or African American'])]

    df = df[df['derived_dwelling_category'] == 'Single Family (1-4 Units):Site-Built']
    df = df.drop(columns = ['derived_dwelling_category'])

    df = df[df['business_or_commercial_purpose'] == 2]
    df = df.drop(columns = ['business_or_commercial_purpose'])

    df = df[df['negative_amortization'] == 2]
    df = df.drop(columns = ['negative_amortization'])

    df = df[df['other_nonamortizing_features'] == 2]
    df = df.drop(columns = ['other_nonamortizing_features'])


    df = df[(df['action_taken'] == 1) |(df['action_taken'] == 2) | (df['action_taken'] == 3)]
    df['action_taken'] = df['action_taken'].astype(int)
    df['action_taken'] = df['action_taken'].replace(3, 0)
    df['action_taken'] = df['action_taken'].replace(2, 1)


    df = df.reset_index(drop=True)
    print(year, "Original: ", df.shape)
    for race in ['White', 'Black or African American']:
        _df = df[df['derived_race'] == race]
        num_split = int(_df.shape[0] * ratio) if ratio is not None else num_samples
        print(year, "Before: ", _df.shape, ratio, num_split)
        if ratio is None:
            ratio = num_samples / _df.shape[0]
        split = StratifiedShuffleSplit(n_splits=1,
                                       test_size=num_split,
                                       random_state=seed)
        for train_index, test_index in split.split(_df, _df['action_taken']):
            tmp = _df.iloc[test_index]
            df_list.append(tmp)
            print(year, "Train After: ", tmp.shape)
            _df = _df.iloc[train_index]
            _df = _df.reset_index(drop=True)
        split = StratifiedShuffleSplit(n_splits=1,
                                       test_size=num_split,
                                       random_state=seed)
        for train_index, test_index in split.split(_df, _df['action_taken']):
            tmp = _df.iloc[test_index]
            df_val_list.append(tmp)
            print(year, "Val After: ", tmp.shape)
            _df = _df.iloc[train_index]
            _df = _df.reset_index(drop=True)
        split = StratifiedShuffleSplit(n_splits=1,
                                       test_size=num_split,
                                       random_state=seed)
        for train_index, test_index in split.split(_df, _df['action_taken']):
            tmp = _df.iloc[test_index]
            df_test_list.append(tmp)
            print(year, "Test After: ", tmp.shape)
            _df = _df.iloc[train_index]
            _df = _df.reset_index(drop=True)


df = pd.concat(df_list)
df_val = pd.concat(df_val_list)
df_test = pd.concat(df_test_list)
print("Final: ", df.shape)

print("Total records: ", N)


2018 (1703559, 99)
2018 Original:  (669976, 32)
2018 Before:  (626583, 32) None 50000
2018 Train After:  (50000, 32)
2018 Val After:  (50000, 32)
2018 Test After:  (50000, 32)
2018 Before:  (43393, 32) 0.07979788790950282 3462
2018 Train After:  (3462, 32)
2018 Val After:  (3462, 32)
2018 Test After:  (3462, 32)
2019 (2137747, 99)
2019 Original:  (837331, 32)
2019 Before:  (785695, 32) 0.07979788790950282 62696
2019 Train After:  (62696, 32)
2019 Val After:  (62696, 32)
2019 Test After:  (62696, 32)
2019 Before:  (51636, 32) 0.07979788790950282 4120
2019 Train After:  (4120, 32)
2019 Val After:  (4120, 32)
2019 Test After:  (4120, 32)
2020 (3440266, 99)
2020 Original:  (1321538, 32)
2020 Before:  (1251372, 32) 0.07979788790950282 99856
2020 Train After:  (99856, 32)
2020 Val After:  (99856, 32)
2020 Test After:  (99856, 32)
2020 Before:  (70166, 32) 0.07979788790950282 5599
2020 Train After:  (5599, 32)
2020 Val After:  (5599, 32)
2020 Test After:  (5599, 32)
2021 (3386558, 99)
2021 Or

In [ ]:
categorical_cols = ['conforming_loan_limit',
       'derived_race', 'applicant_sex', 'co-applicant_sex',
       'applicant_age_above_62', 'co-applicant_age_above_62',
       'loan_type', 'loan_purpose', 'open-end_line_of_credit',
       'occupancy_type',  'applicant_credit_score_type',
        'co-applicant_credit_score_type', 'has_co-applicant']
print(len(categorical_cols))
# categorical conforming_loan_limit derived_dwelling_category derived_race
# applicant_sex co-applicant_sex applicant_age_above_62 co-applicant_age_above_62
# loan_type loan_purpose open-end_line_of_credit business_or_commercial_purpose
# negative_amortization other_nonamortizing_features construction_method occupancy_type
# manufactured_home_secured_property_type manufactured_home_land_property_interest
# applicant_credit_score_type co-applicant_credit_score_type

# create has_co-applicant
# transform co-applicant_sex ()



13


In [ ]:
numeric_cols = ['loan_amount', 'loan_to_value_ratio', 'loan_term', 'property_value', 'income',
                'tract_population', 'tract_minority_population_percent', 'ffiec_msa_md_median_family_income',
                'tract_to_msa_income_percentage', 'tract_owner_occupied_units', 'tract_one_to_four_family_homes',
                'tract_median_age_of_housing_units', 'interest_rate', 'origination_charges', 'total_loan_costs']
print(len(numeric_cols))
ordinal_cols = ['debt_to_income_ratio', 'applicant_age', 'co-applicant_age', 'total_units']
print(len(ordinal_cols))

# numeric loan_amount loan_to_value_ratio loan_term property_value income
# tract_population tract_minority_population_percent ffiec_msa_md_median_family_income
# tract_to_msa_income_percentage tract_owner_occupied_units  tract_one_to_four_family_homes
# tract_median_age_of_housing_units total_units (set 5-24 to 5)
# check loan_to_value_ratio property_value
# order  debt_to_loan_ratio applicant_age co-applicant_age


# transform applicant_age co-applicant_age
# clip loan_amount loan_to_value_ratio property_value income ffiec_msa_md_median_family_income

15
4


In [ ]:
quantiles = {}
for _df, suffix in zip([df, df_val, df_test], ['_train', '_val', '_test']):
    print("suffix", _df.shape)
    _df['has_co-applicant'] = 1
    _df.loc[(_df['co-applicant_sex'] == 5) | (_df['co-applicant_credit_score_type'] == 10), 'has_co-applicant'] = 0
    print(_df['has_co-applicant'].value_counts()/_df.shape[0])

    _df['applicant_sex'] = _df['applicant_sex'].astype('category')
    print(_df['applicant_sex'].value_counts()/_df.shape[0])

    _df.loc[_df['co-applicant_sex'] == 5, 'co-applicant_sex'] = 4
    _df['co-applicant_sex'] = _df['co-applicant_sex'].astype('category')

    print(_df['co-applicant_sex'].value_counts()/_df.shape[0])

    _df['applicant_credit_score_type'] = _df['applicant_credit_score_type'].astype('category')
    print(_df['applicant_credit_score_type'].value_counts()/_df.shape[0])

    _df.loc[_df['co-applicant_credit_score_type'] == 10, 'co-applicant_credit_score_type'] = 9
    _df['co-applicant_credit_score_type'] = _df['co-applicant_credit_score_type'].astype('category')
    print(_df['co-applicant_credit_score_type'].value_counts()/_df.shape[0])



    print("="*50)
    for col in categorical_cols:

        nan_counts = _df[col].isna().sum()
        print(col, nan_counts, _df[col].unique() if nan_counts > 0 else "")

        _df[col] = _df[col].astype('str')
        nan_counts = (_df[col].str.lower() == 'nan').sum()
        print(col, nan_counts, _df[col].unique() if nan_counts > 0 else "")
        _df.loc[_df[col].str.lower() == 'nan', col] = np.nan

        nan_counts = (_df[col].str.lower() == 'exempt').sum()
        print(col, nan_counts, _df[col].unique() if nan_counts > 0 else "")
        _df.loc[_df[col].str.lower() == 'exempt', col] = np.nan

        _df[col] = _df[col].astype('category')
        print("="*50)
    for col in numeric_cols:

        nan_counts = _df[col].isna().sum()
        print(col, "np.nan", nan_counts)

        _df[col] = _df[col].astype('str')
        nan_counts = (_df[col].str.lower() == 'nan').sum()
        print(col, 'nan' , nan_counts)
        _df.loc[_df[col].str.lower() == 'nan', col] = np.nan

        nan_counts = (_df[col].str.lower() == 'Exempt').sum()
        print(col, 'exempt', nan_counts)
        _df.loc[_df[col].str.lower() == 'exempt', col] = np.nan

        _df[col] = pd.to_numeric(_df[col], errors='coerce')
        if col not in quantiles:
            quantiles[col] = []
            lower_bound = _df[col].quantile(0.05)
            upper_bound = _df[col].quantile(0.995)
            quantiles[col].append(lower_bound)
            quantiles[col].append(upper_bound)
        else:
            lower_bound = quantiles[col][0]
            upper_bound = quantiles[col][1]
        _df[col] = _df[col].clip(lower=lower_bound, upper=upper_bound)
        print("="*50)
    _df['debt_to_income_ratio'] = _df['debt_to_income_ratio'].replace('Exempt', np.nan)
    _df['debt_to_income_ratio'] = _df['debt_to_income_ratio'].replace('nan', np.nan)
    _df['debt_to_income_ratio'] = _df['debt_to_income_ratio'].str.split('.').str[0]
    _df.loc[_df['debt_to_income_ratio'].isin([str(num) for num in range(36, 41)]), 'debt_to_income_ratio'] = '36%-<41%'
    _df.loc[_df['debt_to_income_ratio'].isin([str(num) for num in range(41, 45)]), 'debt_to_income_ratio'] = '41%-<45%'
    _df.loc[_df['debt_to_income_ratio'].isin([str(num) for num in range(45, 50)]), 'debt_to_income_ratio'] = '45%-<50%'
    _df['debt_to_income_ratio'] = pd.Categorical(_df['debt_to_income_ratio'],
                                                ordered=True,
                                                categories=['<20%', '20%-<30%',
                                                            '30%-<36%', '36%-<41%',
                                                            '41%-<45%', '45%-<50%',
                                                            '50%-60%', '>60%'])
    print(_df['debt_to_income_ratio'].value_counts()/_df.shape[0])
    _df['applicant_age'] = _df['applicant_age'].replace('8888', np.nan)
    _df['applicant_age'] = _df['applicant_age'].replace('9999', np.nan)
    _df['applicant_age'] = pd.Categorical(_df['applicant_age'],
                                                ordered=True,
                                                categories=['<25', '25-34',
                                                            '35-44', '45-54',
                                                            '55-64', '65-74',
                                                            '>74'])
    print(_df['applicant_age'].value_counts()/_df.shape[0])
    _df['co-applicant_age'] = _df['co-applicant_age'].replace('8888', np.nan)
    _df['co-applicant_age'] = _df['co-applicant_age'].replace('9999', np.nan)
    _df['co-applicant_age'] = pd.Categorical(_df['co-applicant_age'],
                                                ordered=True,
                                                categories=['<25', '25-34',
                                                            '35-44', '45-54',
                                                            '55-64', '65-74',
                                                            '>74'])
    print(_df['co-applicant_age'].value_counts()/_df.shape[0])
    _df['total_units'] = _df['total_units'].astype(str)
    _df['total_units'] = pd.Categorical(_df['total_units'],
                                                ordered=True,
                                                categories=['1', '2', '3', '4',
                                                            '5-24', '25-49', '50-99',
                                                            '100-149', '>149'])
    _df = _df.reset_index(drop=True)

    import pickle
    with open(f"/content/drive/MyDrive/math6912/data/state_CA_cleaned{suffix}_{seed}.pkl", 'wb') as f:
        pickle.dump(_df, f)
    import pickle
    with open(f"/content/drive/MyDrive/math6912/data/state_CA_cleaned{suffix}_{seed}.pkl", 'rb') as f:
        _df = pickle.load(f)



suffix (390541, 32)
has_co-applicant
0    0.524639
1    0.475361
Name: count, dtype: float64
applicant_sex
1    0.667267
2    0.326439
3    0.005364
6    0.000922
4    0.000008
Name: count, dtype: float64
co-applicant_sex
4    0.521863
2    0.346099
1    0.121188
3    0.010386
6    0.000463
Name: count, dtype: float64
applicant_credit_score_type
1       0.288950
2       0.250555
3       0.248622
9       0.112472
8       0.057405
7       0.029221
11      0.009062
5       0.001355
6       0.001209
4       0.001127
1111    0.000023
Name: count, dtype: float64
co-applicant_credit_score_type
9       0.809915
1       0.060391
2       0.051974
3       0.049708
8       0.013804
7       0.010987
11      0.002399
5       0.000499
4       0.000205
6       0.000095
1111    0.000023
Name: count, dtype: float64
conforming_loan_limit 0 
conforming_loan_limit 0 
conforming_loan_limit 0 
derived_race 0 
derived_race 0 
derived_race 0 
applicant_sex 0 
applicant_sex 0 
applicant_sex 0 
co-applicant_sex 